In [2]:
# !pip install spotipy

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Paso 1: Acceder a la API de Spotify

Lo primero que se debe realizar es obtener las credenciales para acceder a la API de [Spotify](https://open.spotify.com/) que nos permitirá extraer data de la plataforma. Para ello es necesario crearse una cuenta en [Spotify for Developers](https://developer.spotify.com/), acceder al [dashboard](https://developer.spotify.com/dashboard/), aceptar los términos y condiciones y darle al botón verde "Create an app". Luego se le da click a la app creada y se usarán las credenciales como en la foto:
<br>
<img src="Resources/Spotify_Credentials.png" width=500px>

In [4]:
# Obtener credenciales con este tutorial https://youtu.be/j4J7B1C3G0U
client_id = '' # Cambiar por tu Client ID
client_secret = '' # Cambiar por tu Client Secret

# Autenticación
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Paso 2: Crear funciones necesarias

<p>1. La función <b>encontrarID</b> encuentra el ID del artista en Spotify usando como input al nombre provisto.</p>

> encontrarID('Arena Hash'): '4AGALQQLo0S46FUlhJ25V0'

<p>2. La función <b>obtenerData</b>, mediante el ID del artista, obtiene la data de los <u><i>álbumes</i></u> (tanto álbum o single), el <u><i>tipo</i></u> de los álbumes, el <u><i>año</i></u> de lanzamiento, las <u><i>canciones</i></u> de los álbumes, los <u><i>artistas</i></u> de las canciones (puede haber colaboraciones), la <u><i>duración</i></u> de las canciones y la <u><i>popularidad</i></u> de las canciones.</p>

> obtenerData('4AGALQQLo0S46FUlhJ25V0'):

| Álbum | Tipo | Año | Canción | Artistas | Duración | Popularidad |
|---|---|---|---|---|---|---|
| Ah Ah Ah | Álbum | 1990 | Y Es Que Sucede Así | Arena Hash | 03:44 | 63 |
| ... | ... | ... | ... | ... | ... | ... |
| Cuando La Cama Me Da Vueltas | Single | 1995 | Cuando La Cama Me Da Vueltas | Arena Hash | 03:04 | 48 |

In [5]:
def encontrarID(artista):
    resultados = sp.search(q=artista, limit=1, type='artist')
    return resultados['artists']['items'][0]['id']

def obtenerData(artista_id): # 'album', 'single', 'appears_on', 'compilation'
    canciones_data = [] # Crear una lista vacía para almacenar los datos de las canciones
    
    albumes = sp.artist_albums(artista_id, album_type='album') # Obtener los álbumes del artista
    singles = sp.artist_albums(artista_id, album_type='single') # Obtener los singles del artista
    
    for album in albumes['items'] + singles['items']: # Obtener las canciones para cada álbum
        album_nombre = album['name']
        album_tipo = {'album':'Álbum', 'single':'Single'}[album['album_type']]
        album_año = album['release_date'].split('-')[0]  # Obtener el año de lanzamiento
        
        tracks = sp.album_tracks(album['id'])

        for track in tracks['items']: # Para cada canción, obtener los detalles y añadirlos a la lista de canciones
            cancion_nombre = track['name']
            cancion_artistas = ', '.join([t['name'] for t in track['artists']])
            cancion_duracion = '{:02d}:{:02d}'.format(*divmod(track['duration_ms'] // 1000, 60))
            cancion_popularidad = sp.track(track['id'])['popularity'] # Obtener la popularidad de la canción
            
            # Añadir los detalles de cadaa canción a la lista de canciones
            canciones_data.append([album_nombre, album_tipo, album_año, cancion_nombre, cancion_artistas, cancion_duracion, cancion_popularidad])
            
    return pd.DataFrame(canciones_data, columns=['Álbum', 'Tipo', 'Año', 'Canción', 'Artistas', 'Duración', 'Popularidad'])

## Paso 3: Obtener ID de artista y su data
El objetivo de este paso es obtener el ID y la data (álbumes, canciones, etc.) de Arena Hash y Pedro Suárez-Vértiz (solista).

#### Paso 3.1: Obtener ID y data de Arena Hash

In [6]:
id_AH = encontrarID('Arena Hash')
print('ID de artista de Arena Hash:', id_AH)

df_AH = obtenerData(id_AH)
df_AH['Etapa'] = 'Arena Hash'
df_AH

ID de artista de Arena Hash: 4AGALQQLo0S46FUlhJ25V0


,Álbum,Tipo,Año,Canción,Artistas,Duración,Popularidad,Etapa
0,Ah Ah Ah,Álbum,1990,Y Es Que Sucede Así,Arena Hash,03:44,66,Arena Hash
1,Ah Ah Ah,Álbum,1990,Materialismo Sexual,Arena Hash,03:55,35,Arena Hash
2,Ah Ah Ah,Álbum,1990,El Túnel del Bus,Arena Hash,04:35,40,Arena Hash
3,Ah Ah Ah,Álbum,1990,El Rey del Ah Ah Ah,Arena Hash,03:51,50,Arena Hash
4,Ah Ah Ah,Álbum,1990,Bésame,Arena Hash,03:41,36,Arena Hash
5,Ah Ah Ah,Álbum,1990,Quiero Vivir Tal Vez Sin Amor,Arena Hash,03:06,33,Arena Hash
6,Ah Ah Ah,Álbum,1990,Murciélago Rodrigo,Arena Hash,03:20,32,Arena Hash
7,Ah Ah Ah,Álbum,1990,Volar Volar,Arena Hash,03:11,31,Arena Hash
8,Ah Ah Ah,Álbum,1990,A Ese Infierno No Voy a Volver,Arena Hash,03:28,43,Arena Hash
9,Ah Ah Ah,Álbum,1990,Cómo Te Va Mi Amor,Arena Hash,05:39,56,Arena Hash


#### Paso 3.2: Obtener ID y data de Pedro Suárez-Vértiz (solista)

In [7]:
id_PSV = encontrarID('Pedro Suárez-Vértiz')
print('ID de artista de Pedro Suárez-Vértiz:', id_PSV)

df_PSV = obtenerData(id_PSV)
df_PSV['Etapa'] = 'Solista'
df_PSV

ID de artista de Pedro Suárez-Vértiz: 3Azj8v21CQ76AVvjuNv5Vg


,Álbum,Tipo,Año,Canción,Artistas,Duración,Popularidad,Etapa
0,Pedro Suárez Vértiz para Bebés (Vol.1),Álbum,2020,Cuando Pienses en Volver,Pedro Suárez-Vértiz,04:07,38,Solista
1,Pedro Suárez Vértiz para Bebés (Vol.1),Álbum,2020,Mi Auto Era una Rana,Pedro Suárez-Vértiz,03:03,33,Solista
2,Pedro Suárez Vértiz para Bebés (Vol.1),Álbum,2020,Me Estoy Enamorando,Pedro Suárez-Vértiz,04:14,31,Solista
3,Pedro Suárez Vértiz para Bebés (Vol.1),Álbum,2020,Sentimiento IncreÍble,Pedro Suárez-Vértiz,03:09,30,Solista
4,Pedro Suárez Vértiz para Bebés (Vol.1),Álbum,2020,Los Globos del Cielo,Pedro Suárez-Vértiz,04:08,29,Solista
...,...,...,...,...,...,...,...,...
115,(No Existen) Técnicas para Olvidar,Álbum,1995,1:38,Pedro Suárez-Vértiz,01:42,36,Solista
116,Amor Yo Te Perdí La Fe,Single,2023,Amor Yo Te Perdí La Fe,Pedro Suárez-Vértiz,02:53,56,Solista
117,Cuando Pienses en Volver (Versión Cumbia- Salsa),Single,2020,Cuando Pienses en Volver (Versión Cumbia- Salsa),"Tony Succar, Pedro Suárez-Vértiz, Daniela Darc...",04:12,51,Solista
118,Jugamos Todos,Single,2019,Jugamos Todos,"Pedro Suárez-Vértiz®La Banda, Pedro Suárez-Vértiz",04:23,43,Solista


#### Paso 3.3: Juntar la data

In [8]:
df = df_AH.append(df_PSV).reset_index(drop=True)
df

,Álbum,Tipo,Año,Canción,Artistas,Duración,Popularidad,Etapa
0,Ah Ah Ah,Álbum,1990,Y Es Que Sucede Así,Arena Hash,03:44,66,Arena Hash
1,Ah Ah Ah,Álbum,1990,Materialismo Sexual,Arena Hash,03:55,35,Arena Hash
2,Ah Ah Ah,Álbum,1990,El Túnel del Bus,Arena Hash,04:35,40,Arena Hash
3,Ah Ah Ah,Álbum,1990,El Rey del Ah Ah Ah,Arena Hash,03:51,50,Arena Hash
4,Ah Ah Ah,Álbum,1990,Bésame,Arena Hash,03:41,36,Arena Hash
...,...,...,...,...,...,...,...,...
128,(No Existen) Técnicas para Olvidar,Álbum,1995,1:38,Pedro Suárez-Vértiz,01:42,36,Solista
129,Amor Yo Te Perdí La Fe,Single,2023,Amor Yo Te Perdí La Fe,Pedro Suárez-Vértiz,02:53,56,Solista
130,Cuando Pienses en Volver (Versión Cumbia- Salsa),Single,2020,Cuando Pienses en Volver (Versión Cumbia- Salsa),"Tony Succar, Pedro Suárez-Vértiz, Daniela Darc...",04:12,51,Solista
131,Jugamos Todos,Single,2019,Jugamos Todos,"Pedro Suárez-Vértiz®La Banda, Pedro Suárez-Vértiz",04:23,43,Solista


## Paso 4: Gráficas

### Paso 4.1: Canciones

#### Paso 4.1.0: Tabla de canciones

In [9]:
df_canciones = df.copy()
df_canciones['Duración_seg'] = df_canciones['Duración'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))
df_canciones.head()

,Álbum,Tipo,Año,Canción,Artistas,Duración,Popularidad,Etapa,Duración_seg
0,Ah Ah Ah,Álbum,1990,Y Es Que Sucede Así,Arena Hash,03:44,66,Arena Hash,224
1,Ah Ah Ah,Álbum,1990,Materialismo Sexual,Arena Hash,03:55,35,Arena Hash,235
2,Ah Ah Ah,Álbum,1990,El Túnel del Bus,Arena Hash,04:35,40,Arena Hash,275
3,Ah Ah Ah,Álbum,1990,El Rey del Ah Ah Ah,Arena Hash,03:51,50,Arena Hash,231
4,Ah Ah Ah,Álbum,1990,Bésame,Arena Hash,03:41,36,Arena Hash,221


#### Paso 4.1.1: Canciones más populares

In [10]:
# df_canciones.sort_values(by=['Popularidad'], ascending=False).head(10)

In [16]:
canciones_populares = df_canciones.sort_values(by=['Popularidad'], ascending=False).head(10)

color_dict = {'Solista': '#1ED660', 'Arena Hash': 'white'} 

# Creamos el gráfico de barras
fig = px.bar(canciones_populares, 
             x='Canción', 
             y='Popularidad',
             text='Popularidad',
             color='Etapa',
             color_discrete_map=color_dict,
             hover_data={'Álbum':True, 'Año':True},
             title='Canciones más populares',
             template='plotly_dark') 

# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Canciones más populares</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  xaxis=dict(categoryorder='total descending', title_font=dict(size=18), tickfont=dict(size=18)),
                  yaxis=dict(title_font=dict(size=18)))

# Establecemos el rango del eje Y
fig.update_yaxes(range=[55, canciones_populares['Popularidad'].max()+1], showgrid=False, showticklabels=False)

# Ajustar posición y formato del texto
fig.update_traces(textposition='outside', textfont_size=18)

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_PSV_Spotify.png",
                          xref="paper", yref="paper",x=0.94, y=1.12,
                          sizex=0.2, sizey=0.2,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()

#### Paso 4.1.2: Canciones con más de duración

In [11]:
# df_canciones.sort_values(by=['Duración'], ascending=False).head(10)

In [17]:
canciones_duracion = df_canciones.sort_values(by=['Duración'], ascending=False).head(10)

color_dict = {'Solista': '#1ED660', 'Arena Hash': 'white'} 

# Creamos el gráfico de barras
fig = px.bar(canciones_duracion, 
             x='Canción', 
             y='Duración_seg',
             text='Duración',
             color='Etapa',
             color_discrete_map=color_dict,
             hover_data={'Año':True, 'Tipo':True, 'Álbum':True},
             title='Canciones con más duración',
             template='plotly_dark')

# Ajustar posición y formato del texto
fig.update_traces(textposition='outside', textfont_size=18)

# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Canciones con más duración</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  xaxis=dict(categoryorder='total descending', title_font=dict(size=18), tickfont=dict(size=18)),
                  yaxis=dict(title_font=dict(size=18)))

fig.update_yaxes(range=[250, canciones_duracion['Duración_seg'].max()+10], showgrid=False, showticklabels=False,
                title_text='Duración (min)')

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_PSV_Spotify.png",
                          xref="paper", yref="paper",x=0.94, y=1.12,
                          sizex=0.2, sizey=0.2,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()

#### Paso 4.1.3: Canciones en Álbumes vs. Singles

In [13]:
# df_canciones.groupby(['Tipo','Etapa']).agg({'Canción':'count'}).reset_index()

In [14]:
canciones_tipo = df_canciones.groupby(['Tipo','Etapa']).agg({'Canción':'count'}).reset_index()

color_dict = {'Solista': '#1ED660', 'Arena Hash': 'white'} 

# Creamos el gráfico de barras
fig = px.bar(canciones_tipo, 
             x='Tipo', 
             y='Canción',
             text='Canción',
             color='Etapa',
             color_discrete_map=color_dict,
             labels={'Canción':'Canciones'},
             title='Popularidad Promedio de las Canciones por Álbum',
             template='plotly_dark')

# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Álbumes vs. Singles</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  xaxis={'categoryorder': 'total descending'},
                  yaxis={'categoryorder': 'total descending'})

# Establecemos el rango del eje Y
fig.update_yaxes(showgrid=False, showticklabels=False)

# Ajustar posición y formato del texto
fig.update_traces(textposition='outside', insidetextanchor='middle', textfont=dict(size=18),
                  width=0.4)

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_PSV_Spotify.png",
                          xref="paper", yref="paper",x=0.94, y=1.12,
                          sizex=0.17, sizey=0.17,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()

### Paso 4.2: Álbumes

#### Paso 4.2.0: Tabla de álbumes

In [19]:
df_albumes = df.copy()
df_albumes['Duración_seg'] = df_albumes['Duración'].apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]))
df_albumes = df_albumes.groupby('Álbum').agg({'Tipo':'min', 'Año':'min', 'Popularidad':'mean', 'Duración_seg':'sum', 'Canción':'count', 'Etapa':'min'})
df_albumes['Duración'] = df_albumes['Duración_seg'].apply(lambda x:'{:02d}:{:02d}'.format(*divmod(int(x), 60)))
df_albumes.head()

,Tipo,Año,Popularidad,Duración_seg,Canción,Etapa,Duración
Álbum,,,,,,,
(No Existen) Técnicas para Olvidar,Álbum,1995,52.818182,2240,11,Solista,37:20
Ah Ah Ah,Álbum,1990,42.200000,2310,10,Arena Hash,38:30
Amazonas,Álbum,2009,38.000000,3110,12,Solista,51:50
Amazonas (Uncut),Álbum,2010,30.666667,4368,18,Solista,72:48
Amor Yo Te Perdí La Fe,Single,2023,56.000000,173,1,Solista,02:53


#### Paso 4.2.1: Álbumes más populares

In [20]:
# df_albumes.sort_values(by=['Popularidad'], ascending=False).reset_index()

In [35]:
albumes_populares = df_albumes.sort_values(by=['Popularidad'], ascending=False).reset_index().head(10)

color_dict = {'Solista': '#1ED660', 'Arena Hash': 'white'} 

# Creamos el gráfico de barras
fig = px.bar(albumes_populares, 
             x='Álbum', 
             y='Popularidad',
             text='Popularidad',
             color='Etapa',
             color_discrete_map=color_dict,
             hover_data={'Año':True},
             title='Álbumes más populares',
             template='plotly_dark')

# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Álbumes más populares</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  xaxis=dict(categoryorder='total descending', title_font=dict(size=18), tickfont=dict(size=18)),
                  yaxis=dict(title_font=dict(size=18)), height=700)

# Establecemos el rango del eje Y
fig.update_yaxes(range=[20, albumes_populares['Popularidad'].max()+4], showgrid=False, showticklabels=False)

# Ajustar posición y formato del texto
fig.update_traces(texttemplate='%{text:.1f}', textposition='outside', textfont_size=18)

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_PSV_Spotify.png",
                          xref="paper", yref="paper",x=0.94, y=1.12,
                          sizex=0.25, sizey=0.25,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()

#### Paso 4.2.2: Álbumes más duración

In [22]:
# albumes_duracion = df_albumes.sort_values(by=['Duración_seg'], ascending=False).reset_index()
# albumes_duracion['Duración_prom_seg'] = albumes_duracion['Duración_seg'] / albumes_duracion['Canción']
# albumes_duracion['Duración_prom'] = albumes_duracion['Duración_prom_seg'].apply(lambda x:'{:02d}:{:02d}'.format(*divmod(int(x), 60)))
# albumes_duracion

In [54]:
albumes_duracion = df_albumes.sort_values(by=['Duración_seg'], ascending=False).reset_index().head(10)
albumes_duracion['Duración_prom_seg'] = albumes_duracion['Duración_seg'] / albumes_duracion['Canción']
albumes_duracion['Duración_prom'] = albumes_duracion['Duración_prom_seg'].apply(lambda x:'{:02d}:{:02d}'.format(*divmod(int(x), 60)))

color_dict = {'Solista': '#1ED660', 'Arena Hash': 'white'} 

# Creamos el gráfico de barras
fig = px.bar(albumes_duracion, 
             x='Álbum', 
             y='Duración_seg',
             text='Duración',
             color='Etapa',
             color_discrete_map=color_dict,
             labels={'Duración_seg':'Duración'},
             hover_data={'Año':True, 'Tipo':True},
             title='Álbumes con más duración',
             template='plotly_dark')

# Ajustar posición y formato del texto
fig.update_traces(textposition='outside', textfont_size=18)

# Añadir un gráfico de líneas con la duración promedio en el eje y secundario
fig.add_trace(go.Scatter(x=albumes_duracion['Álbum'],
                         y=albumes_duracion['Duración_prom_seg'],
                         mode='lines+markers',
                         yaxis='y2',
                         name='Duración Promedio',
                         line=dict(color='#F3E5AB'),
                         text='Duración Promedio= ' + albumes_duracion['Duración_prom'],
                         hoverinfo='text'))

# Establecemos el rango del eje Y
fig.update_yaxes(range=[0, albumes_duracion['Duración_seg'].max()+400])



# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Álbumes con más duración</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  xaxis=dict(categoryorder='total descending', title_font=dict(size=18), tickfont=dict(size=18)),
                  yaxis=dict(title_font=dict(size=18)), height=700,
                  yaxis2=dict(title='Duración promedio',
                              overlaying='y',
                              side='right',
                              range=[150, albumes_duracion['Duración_prom_seg'].max()+150]))

# Establecemos el rango del eje Y
fig.update_yaxes(showgrid=False, showticklabels=False, title='')
fig.update_xaxes(tickangle=60)

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_PSV_Spotify.png",
                          xref="paper", yref="paper",x=0.94, y=1.12,
                          sizex=0.25, sizey=0.25,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()

#### Paso 4.2.3: Álbumes con más canciones

In [24]:
# df_albumes[df_albumes['Tipo']=='Álbum'].sort_values(by=['Canción'], ascending=False).reset_index().head(10)

In [58]:
albumes_canciones = df_albumes[df_albumes['Tipo']=='Álbum'].sort_values(by=['Canción'], ascending=False).reset_index().head(10)

color_dict = {'Solista': '#1ED660', 'Arena Hash': 'white'} 

# Creamos el gráfico de barras
fig = px.bar(albumes_canciones, 
             x='Álbum', 
             y='Canción',
             text='Canción',
             color='Etapa',
             color_discrete_map=color_dict,
             labels={'Canción':'# Canciones'},
             hover_data={'Año':True},
             title='Álbumes con más canciones',
             template='plotly_dark')

# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Álbumes con más canciones</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  xaxis=dict(categoryorder='total descending', title_font=dict(size=18), tickfont=dict(size=18)),
                  yaxis=dict(title_font=dict(size=18)), height=700)

# Establecemos el rango del eje Y
fig.update_yaxes(range=[5, albumes_canciones['Canción'].max()+2], showgrid=False, showticklabels=False)

# Ajustar posición y formato del texto
fig.update_traces(textposition='outside', textfont_size=18)

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_PSV_Spotify.png",
                          xref="paper", yref="paper",x=0.94, y=1.12,
                          sizex=0.22, sizey=0.22,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()

#### Paso 4.2.4: Álbumes vs. Singles

In [26]:
# df_albumes.reset_index().groupby(['Tipo','Etapa']).agg({'Álbum':'count'}).reset_index()

In [27]:
albumes_singles = df_albumes.reset_index().groupby(['Tipo','Etapa']).agg({'Álbum':'count'}).reset_index()

color_dict = {'Solista': '#1ED660', 'Arena Hash': 'white'} 

# Creamos el gráfico de barras
fig = px.bar(albumes_singles, 
             x='Tipo', 
             y='Álbum',
             text='Álbum',
             color='Etapa',
             color_discrete_map=color_dict,
             labels={'Álbum':'Cantidad'},
             title='Álbumes vs. Singles',
             template='plotly_dark')

# Ordenamos las categorías del eje X de acuerdo a la popularidad
fig.update_layout(title={'text': '<i>Álbumes vs. Singles</i>',
                         'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top',
                         'font':dict(size=25, color='white')},
                  xaxis={'categoryorder': 'total descending'},
                  yaxis={'categoryorder': 'total descending'})

# Establecemos el rango del eje Y
fig.update_yaxes(showgrid=False, showticklabels=False)

# Ajustar posición y formato del texto
fig.update_traces(textposition='inside', insidetextanchor='middle', textfont=dict(size=18),
                  width=0.4)

# Añadimos la imagen
fig.add_layout_image(dict(source="Resources/Logo_PSV_Spotify.png",
                          xref="paper", yref="paper",x=0.94, y=1.12,
                          sizex=0.17, sizey=0.17,
                          xanchor="center", yanchor="middle",
                          layer="above"))

fig.show()

#### Paso 4.2.5: Álbumes vs. tiempo

In [28]:
df_albumes.sort_values(by=['Año','Etapa'], ascending=False).reset_index()[['Álbum','Año','Etapa']]

,Álbum,Año,Etapa
0,Amor Yo Te Perdí La Fe,2023,Solista
1,Como Te Va Mi Amor,2023,Arena Hash
2,Cuando Pienses en Volver (Versión Cumbia- Salsa),2020,Solista
3,Pedro Suárez Vértiz para Bebés (Vol.1),2020,Solista
4,Jugamos Todos,2019,Solista
5,El Triunfo Tan Soñado,2017,Solista
6,Amazonas (Uncut),2010,Solista
7,Amazonas,2009,Solista
8,Pedro Suárez-Vértiz,2007,Solista
9,Talk Show,2007,Solista
